# Bronze layer
---

#### Reading all files into Dataframe

In [0]:
all_days_df = spark.read.format('json').option('header','true').load("dbfs:/FileStore/tables/GHarchive/*.json.gz")

#### Filtering for dates January 16-31

In [0]:
from pyspark.sql.functions import current_timestamp, col, dayofmonth
from pyspark.sql.types import DateType

df = all_days_df.withColumn('created_at_date', col('created_at').cast(DateType()))

In [0]:
df = df.filter((dayofmonth(col('created_at')) >= 16))

#### Adding Timestamp

In [0]:
from pyspark.sql.functions import current_timestamp, date_format

df = df.withColumn('Date', current_timestamp())

#### Connecting to ADLS

In [0]:
container_name = "team3-project2"
storage_account_name = "20231023desa"
conn_str = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/BronzeLayer"
access_key = "1PuUNqP+64imQ7kbw5IFioViCAYOQjC18P1tZlFZaN/r1MME7Wgzh+8vbNf1QZg3rCLD6Femlsby+AStdXYJ+g=="

spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", #set azure account key
    access_key # to this access key
)

### Writing to ADLS in 128mb partitions

In [0]:
df.repartition(18).write.format('parquet').option('header', 'true').partitionBy('created_at_date').mode('overwrite').save(conn_str)